In [1]:
!pip install --upgrade --quiet  langchain-core langchain-community langchain-openai grandalf

<a href="https://colab.research.google.com/github/qxr777/llm-application-code/blob/master/langchain_expression_language/parallelism-example.ipynb" target="_parent">Open In Colab</a>

In [2]:
# 三选一 加载OPENAI_API_KEY

# 本地运行，包含.env文件
from dotenv import load_dotenv
load_dotenv()

# 本地运行，不包含.env文件
# import os
# os.environ['OPENAI_API_KEY'] = 'sk-123456789'

# google colab运行，需要配置secret key
# import os
# from google.colab import userdata
# os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')

True

In [3]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableParallel

model = ChatOpenAI()
chain1 = ChatPromptTemplate.from_template("tell me a joke about {topic}") | model
chain2 = (
    ChatPromptTemplate.from_template("write a short (2 line) poem about {topic}")
    | model
)
combined = RunnableParallel(joke=chain1, poem=chain2)

# 打印chain的计算图
combined.get_graph().print_ascii()

               +--------------------------+                
               | Parallel<joke,poem>Input |                
               +--------------------------+                
                   ***               ***                   
                ***                     ***                
              **                           **              
+--------------------+              +--------------------+ 
| ChatPromptTemplate |              | ChatPromptTemplate | 
+--------------------+              +--------------------+ 
           *                                   *           
           *                                   *           
           *                                   *           
    +------------+                      +------------+     
    | ChatOpenAI |                      | ChatOpenAI |     
    +------------+*                     +------------+     
                   ***               ***                   
                      ***         ***   

In [4]:
%%time
chain1.invoke({"topic": "bears"})

CPU times: user 42.8 ms, sys: 17.7 ms, total: 60.5 ms
Wall time: 1.35 s


AIMessage(content='Why did the bear bring a flashlight to the party? \nBecause he heard it was going to be a bear-y bright affair!', response_metadata={'token_usage': {'completion_tokens': 26, 'prompt_tokens': 13, 'total_tokens': 39}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-fa071f42-0d58-41df-802c-970648254030-0', usage_metadata={'input_tokens': 13, 'output_tokens': 26, 'total_tokens': 39})

In [5]:
%%time
chain2.invoke({"topic": "bears"})

CPU times: user 13.6 ms, sys: 2.99 ms, total: 16.6 ms
Wall time: 987 ms


AIMessage(content='Majestic creatures of the wild,\nIn their strength and beauty, they beguile.', response_metadata={'token_usage': {'completion_tokens': 19, 'prompt_tokens': 17, 'total_tokens': 36}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-ceb0b980-9fdc-41a5-a498-eb02d9b8820c-0', usage_metadata={'input_tokens': 17, 'output_tokens': 19, 'total_tokens': 36})

In [6]:
%%time
combined.invoke({"topic": "bears"})

CPU times: user 34.2 ms, sys: 6.63 ms, total: 40.8 ms
Wall time: 1.13 s


{'joke': AIMessage(content='Why did the bear break up with his girlfriend?\nBecause she was unbearable!', response_metadata={'token_usage': {'completion_tokens': 15, 'prompt_tokens': 13, 'total_tokens': 28}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-5b76d595-13d5-48be-9d5f-0bc1b9dd8179-0', usage_metadata={'input_tokens': 13, 'output_tokens': 15, 'total_tokens': 28}),
 'poem': AIMessage(content='Gentle giants of the forest,\nBears roam with strength and grace.', response_metadata={'token_usage': {'completion_tokens': 16, 'prompt_tokens': 17, 'total_tokens': 33}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-11073e5d-4031-4c07-a5ce-858f3f32f239-0', usage_metadata={'input_tokens': 17, 'output_tokens': 16, 'total_tokens': 33})}